# Load Profiles

Get standard load profiles from https://www.bdew.de/energie/standardlastprofile-strom/.
Download .zip folder, we use only the Repräsentative Profile VDEW here, renamed to
representative_profiles_vdew.xls.

May query personal load profile from WN with library ...

In [ ]:
from pathlib import Path
import datetime

import holidays
import numpy as np
import pandas as pd

In [ ]:
lastprofile_file = Path('.') / 'external' / 'Lastprofile' / 'representative_profiles_vdew.xls'
assert lastprofile_file.is_file(), f'Did not find file with representative load profiles at {lastprofile_file}'

In [ ]:
def get_profile_from_file(fp: Path, profile: str = 'H0') -> pd.DataFrame:
    """From the VDEW profiles xls file, get one sheet.

    Args:
        fp (Path): Path to .xls file.
        profile (str): Profile name corresponding to the sheet name in the file.
            Defaults to H0.

    Returns:
        pd.DataFrame: DataFrame with column Multiindex Season
            (Winter, Sommer, Übergangszeit) and Daytype (Samstag, Sonntag, Werktag),
            and Index Time from 0:15 to 24:00 in 15 minute intervals.
    """
    df = pd.read_excel(fp, sheet_name=profile, header=[1,2], index_col=0, nrows=24*4)

    return df

In [ ]:
i = pd.Index(['00:15', '00:30'])
i = i.map(lambda x: time.fromisoformat(x))

In [ ]:
h0_profiles = get_profile_from_file(lastprofile_file, 'H0')

In [ ]:
# Constants
AT_HOLIDAYS = holidays.AUT()

# Dynamization function coefficients
A4=-3.92e-10
A3=0.00000032
A2=-0.0000702
A1=0.0021
A0=1.24

In [ ]:
def dynamize_day(ser: pd.Series, dayofyear: int) -> pd.Series:
    """Dynamize profile 15 minute series by dayofyear."""
    dynamization_value = (
        np.power(dayofyear, 4) * A4
        + np.power(dayofyear, 3) * A3
        + np.power(dayofyear, 2) * A2
        + dayofyear * A1
        + A0
    )
    return ser * dynamization_value

In [ ]:
def load_profile_for_day(day: datetime.date, profile_df: pd.DataFrame, dynamize: bool=False) -> pd.Series:
    """Get the 15 minute load profile for a single day.

    Args:
        day (datetime.date): Date for which to get the load profile.
        profile_df (pd.DataFrame): In the format as given by get_profile_from_file.
        dynamize (bool): Should dynamization by the 4th-grade polynomial dynamization
            function for day of year be applied. This should be only done for H0. Defaults to False.

    Returns:
        pd.Series: Series with Watt values every 15 minutes, and a pd.DatetimeIndex.
    """
    if (day <= datetime.date(day.year, 3, 20)) or (day >= datetime.date(day.year, 11, 1)):
        season = 'Winter'
    elif (day >= datetime.date(day.year, 5, 15)) and (day <= datetime.date(day.year, 9, 14)):
        season =  'Sommer'
    else:
        season = "Übergangszeit"
        
    if (day in AT_HOLIDAYS) or (day.isoweekday == 7):
        weekday = 'Sonntag'
    elif (day.isoweekday == 6) or (day in [datetime.date(day.year, 12, 24), datetime.date(day.year, 12, 31)]):
        weekday = 'Samstag'
    else:
        weekday = 'Werktag'

    df = profile_df[season][weekday]
    df.index = df.index.map(lambda time: datetime.datetime.combine(day, time))

    if dynamize:
        df = dynamize_day(df, df.index.dayofyear[0])

    return df

In [ ]:
datetime.date(2020, 1, 4).day

In [ ]:
df = load_profile_for_day(datetime.date(2020, 5, 30), h0_profiles, True)

dayofyear = df.index.dayofyear[0]

In [ ]:
# Make function to get load profile for date from to